In [22]:
# https://colab.research.google.com/drive/1vMkH8LmiCCOiCo4KTTEcv-NU8_OGn0ie?usp=sharing#scrollTo=s3ucr4kRmBKk

In [49]:
# torch==1.11.0
import warprnnt_pytorch
import torch
import numpy as np

In [50]:
device = torch.device('cuda', 0)

In [51]:
logits = torch.FloatTensor([[[[0.1, 0.6, 0.1, 0.1, 0.1],
                              [0.1, 0.1, 0.6, 0.1, 0.1],
                              [0.1, 0.1, 0.2, 0.8, 0.1]],
                              [[0.1, 0.6, 0.1, 0.1, 0.1],
                              [0.1, 0.1, 0.2, 0.1, 0.1],
                              [0.7, 0.1, 0.2, 0.1, 0.1]]]])
warp_transducer_logits = logits.clone()
torchaudio_logits = logits.clone()
optimized_transducer_logits = logits.clone()

logits_cuda = logits.to(device)
warp_transducer_logits_cuda = logits_cuda.clone()
torchaudio_logits_cuda = logits_cuda.clone()
optimized_transducer_logits_cuda = logits_cuda.clone()


targets = torch.tensor([[1, 2]], dtype=torch.int32)
logit_lengths = torch.tensor([2], dtype=torch.int32)
target_lengths = torch.tensor([2], dtype=torch.int32)

targets_cuda = targets.to(device)
logit_lengths_cuda = logit_lengths.to(device)
target_lengths_cuda = target_lengths.to(device)

In [52]:
print(logits.shape)
print(logit_lengths.shape)
print(targets.shape)
print(target_lengths.shape)

torch.Size([1, 2, 3, 5])
torch.Size([1])
torch.Size([1, 2])
torch.Size([1])


In [53]:
warp_transducer_logits.requires_grad_(True)
torchaudio_logits.requires_grad_(True)
optimized_transducer_logits.requires_grad_(True)

warp_transducer_logits_cuda.requires_grad_(True)
torchaudio_logits_cuda.requires_grad_(True)
optimized_transducer_logits_cuda.requires_grad_(True)

tensor([[[[0.1000, 0.6000, 0.1000, 0.1000, 0.1000],
          [0.1000, 0.1000, 0.6000, 0.1000, 0.1000],
          [0.1000, 0.1000, 0.2000, 0.8000, 0.1000]],

         [[0.1000, 0.6000, 0.1000, 0.1000, 0.1000],
          [0.1000, 0.1000, 0.2000, 0.1000, 0.1000],
          [0.7000, 0.1000, 0.2000, 0.1000, 0.1000]]]], device='cuda:0',
       requires_grad=True)

# warp_transducer

In [54]:
warp_transducer_cpu_loss = warprnnt_pytorch.rnnt_loss(warp_transducer_logits, 
                                                      targets, 
                                                      logit_lengths, 
                                                      target_lengths,
                                                      blank=0,
                                                      reduction='mean',
                                                      fastemit_lambda=0)

warp_transducer_cuda_loss = warprnnt_pytorch.rnnt_loss(warp_transducer_logits_cuda, 
                                                       targets_cuda, 
                                                       logit_lengths_cuda, 
                                                       target_lengths_cuda,
                                                       blank=0,
                                                       reduction='mean',
                                                       fastemit_lambda=0)

In [55]:
print(f'warp_transducer, cpu_loss: {warp_transducer_cpu_loss}, cuda_loss: {warp_transducer_cuda_loss}')

warp_transducer, cpu_loss: tensor([4.4957], grad_fn=<_RNNTBackward>), cuda_loss: tensor([4.4957], device='cuda:0', grad_fn=<_RNNTBackward>)


In [56]:
np.testing.assert_allclose(warp_transducer_cpu_loss.detach().numpy(), warp_transducer_cuda_loss.cpu().detach().numpy(), rtol=1e-7)

In [57]:
warp_transducer_cpu_loss.backward()
warp_transducer_cuda_loss.backward()

In [58]:
print(warp_transducer_logits.grad)
print(warp_transducer_logits.grad.shape)
print(warp_transducer_logits.grad.device)

tensor([[[[-0.1312, -0.3999,  0.1770,  0.1770,  0.1770],
          [-0.1857,  0.1225, -0.1817,  0.1225,  0.1225],
          [-0.3209,  0.0627,  0.0693,  0.1262,  0.0627]],

         [[ 0.0546, -0.2182,  0.0546,  0.0546,  0.0546],
          [ 0.1207,  0.1207, -0.4830,  0.1207,  0.1207],
          [-0.6926,  0.1687,  0.1865,  0.1687,  0.1687]]]])
torch.Size([1, 2, 3, 5])
cpu


In [59]:
print(warp_transducer_logits_cuda.grad)
print(warp_transducer_logits_cuda.grad.device)

tensor([[[[-0.1312, -0.3999,  0.1770,  0.1770,  0.1770],
          [-0.1857,  0.1225, -0.1817,  0.1225,  0.1225],
          [-0.3209,  0.0627,  0.0693,  0.1262,  0.0627]],

         [[ 0.0546, -0.2182,  0.0546,  0.0546,  0.0546],
          [ 0.1207,  0.1207, -0.4830,  0.1207,  0.1207],
          [-0.6926,  0.1687,  0.1865,  0.1687,  0.1687]]]], device='cuda:0')
cuda:0


In [60]:
np.testing.assert_allclose(warp_transducer_logits.grad.numpy(), warp_transducer_logits_cuda.grad.cpu().numpy(), rtol=1e-6)

# torchaudio

In [13]:
!pip install --upgrade pip 
# !pip uninstall torchaudio -y
!TMPDIR=$PWD pip install torchaudio==0.11.0 -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 603.7 kB/s eta 0:00:0000:0100:01


In [61]:
import torchaudio

In [62]:
torchaudio_cpu_loss = torchaudio.functional.rnnt_loss(torchaudio_logits,
                                                      targets,
                                                      logit_lengths,
                                                      target_lengths,
                                                      blank=0,
                                                      reduction='mean'
                                                      )

torchaudio_cuda_loss = torchaudio.functional.rnnt_loss(torchaudio_logits_cuda,
                                                       targets_cuda,
                                                       logit_lengths_cuda,
                                                       target_lengths_cuda,
                                                       blank=0,
                                                       reduction='mean'
                                                       )

In [63]:
print(f'torchaudio, cpu_loss: {torchaudio_cpu_loss}, cuda_loss: {torchaudio_cuda_loss}')

torchaudio, cpu_loss: 4.495666980743408, cuda_loss: 4.49566650390625


In [64]:
np.testing.assert_allclose(torchaudio_cpu_loss.detach().numpy(), torchaudio_cuda_loss.cpu().detach().numpy(), rtol=1e-6)

In [65]:
torchaudio_cpu_loss.backward()
torchaudio_cuda_loss.backward()

In [66]:
print(torchaudio_logits.grad)

tensor([[[[-0.1312, -0.3999,  0.1770,  0.1770,  0.1770],
          [-0.1857,  0.1225, -0.1817,  0.1225,  0.1225],
          [-0.3209,  0.0627,  0.0693,  0.1262,  0.0627]],

         [[ 0.0546, -0.2182,  0.0546,  0.0546,  0.0546],
          [ 0.1207,  0.1207, -0.4830,  0.1207,  0.1207],
          [-0.6926,  0.1687,  0.1865,  0.1687,  0.1687]]]])


In [67]:
print(torchaudio_logits_cuda.grad)

tensor([[[[-0.1312, -0.3999,  0.1770,  0.1770,  0.1770],
          [-0.1857,  0.1225, -0.1817,  0.1225,  0.1225],
          [-0.3209,  0.0627,  0.0693,  0.1262,  0.0627]],

         [[ 0.0546, -0.2182,  0.0546,  0.0546,  0.0546],
          [ 0.1207,  0.1207, -0.4830,  0.1207,  0.1207],
          [-0.6926,  0.1687,  0.1865,  0.1687,  0.1687]]]], device='cuda:0')


In [68]:
np.testing.assert_allclose(torchaudio_logits.grad.numpy(), torchaudio_logits_cuda.grad.cpu().numpy(), rtol=1e-6)

# check warp transducer with torchaudio 

In [69]:
np.testing.assert_allclose(warp_transducer_cpu_loss.detach().numpy(), torchaudio_cuda_loss.cpu().detach().numpy(), rtol=1e-7)

In [70]:
np.testing.assert_allclose(warp_transducer_logits.grad.numpy(), torchaudio_logits.grad.numpy(), rtol=1e-6)

In [71]:
np.testing.assert_allclose(warp_transducer_logits_cuda.grad.cpu().numpy(), torchaudio_logits_cuda.grad.cpu().numpy(), rtol=1e-6)

# optimized_transducer
# https://github.com/csukuangfj/optimized_transducer

In [77]:
!pip uninstall optimized_transducer -y

Found existing installation: optimized-transducer 1.4
Uninstalling optimized-transducer-1.4:
  Successfully uninstalled optimized-transducer-1.4


In [78]:
!TMPDIR=$PWD OT_CMAKE_ARGS="-DOT_COMPUTE_ARCHS=37 -DCMAKE_BUILD_TYPE=Release" OT_MAKE_ARGS="-j" pip install --verbose optimized_transducer

Using pip 22.3.1 from /workspace/DeepSpeech-2.x/speechx/venv/lib/python3.7/site-packages/pip (python 3.7)
  Using cached optimized_transducer-1.4.tar.gz (46 kB)
  Running command python setup.py egg_info
  running egg_info
  creating /workspace/rnnt/warp-transducer-espnet/pip-pip-egg-info-1hltfrz7/optimized_transducer.egg-info
  writing /workspace/rnnt/warp-transducer-espnet/pip-pip-egg-info-1hltfrz7/optimized_transducer.egg-info/PKG-INFO
  writing dependency_links to /workspace/rnnt/warp-transducer-espnet/pip-pip-egg-info-1hltfrz7/optimized_transducer.egg-info/dependency_links.txt
  writing top-level names to /workspace/rnnt/warp-transducer-espnet/pip-pip-egg-info-1hltfrz7/optimized_transducer.egg-info/top_level.txt
  writing manifest file '/workspace/rnnt/warp-transducer-espnet/pip-pip-egg-info-1hltfrz7/optimized_transducer.egg-info/SOURCES.txt'
  reading manifest file '/workspace/rnnt/warp-transducer-espnet/pip-pip-egg-info-1hltfrz7/optimized_transducer.egg-info/SOURCES.txt'
  readi

In [80]:
import optimized_transducer

In [81]:
optimized_transducer_cpu_loss = optimized_transducer.transducer_loss(optimized_transducer_logits.reshape(-1, logits.size(-1)),
                                                                    targets,
                                                                    logit_lengths,
                                                                    target_lengths,
                                                                    blank=0,
                                                                    from_log_softmax=False,
                                                                    reduction='mean'
                                                                    )
optimized_transducer_cuda_loss = optimized_transducer.transducer_loss(optimized_transducer_logits_cuda.reshape(-1, logits.size(-1)),
                                                                     targets_cuda,
                                                                     logit_lengths_cuda,
                                                                     target_lengths_cuda,
                                                                     blank=0,
                                                                     from_log_softmax=False,
                                                                     reduction='mean'
                                                                     )

In [82]:
print(f'optimized_transducer, cpu_loss: {optimized_transducer_cpu_loss}, cuda_loss: {optimized_transducer_cuda_loss}')

optimized_transducer, cpu_loss: 4.495666980743408, cuda_loss: 4.495666980743408


In [84]:
np.testing.assert_allclose(optimized_transducer_cpu_loss.detach().numpy(), optimized_transducer_cuda_loss.cpu().detach().numpy(), rtol=1e-7)

In [85]:
optimized_transducer_cpu_loss.backward()
optimized_transducer_cuda_loss.backward()

In [86]:
print(optimized_transducer_logits.grad)
print(optimized_transducer_logits_cuda.grad)

tensor([[[[-0.1312, -0.3999,  0.1770,  0.1770,  0.1770],
          [-0.1857,  0.1225, -0.1817,  0.1225,  0.1225],
          [-0.3209,  0.0627,  0.0693,  0.1262,  0.0627]],

         [[ 0.0546, -0.2182,  0.0546,  0.0546,  0.0546],
          [ 0.1207,  0.1207, -0.4830,  0.1207,  0.1207],
          [-0.6926,  0.1687,  0.1865,  0.1687,  0.1687]]]])
tensor([[[[-0.1312, -0.3999,  0.1770,  0.1770,  0.1770],
          [-0.1857,  0.1225, -0.1817,  0.1225,  0.1225],
          [-0.3209,  0.0627,  0.0693,  0.1262,  0.0627]],

         [[ 0.0546, -0.2182,  0.0546,  0.0546,  0.0546],
          [ 0.1207,  0.1207, -0.4830,  0.1207,  0.1207],
          [-0.6926,  0.1687,  0.1865,  0.1687,  0.1687]]]], device='cuda:0')


In [91]:
np.testing.assert_allclose(optimized_transducer_logits.grad.numpy(), optimized_transducer_logits_cuda.grad.cpu().numpy(), rtol=1e-6)

In [92]:
assert torch.allclose(torchaudio_logits.grad, optimized_transducer_logits.grad)
assert torch.allclose(torchaudio_logits_cuda.grad, optimized_transducer_logits_cuda.grad)
assert torch.allclose(torchaudio_logits.grad, optimized_transducer_logits_cuda.grad.cpu())